# Transfer learning - Export Keras model for Google Cloud ML API


8/2/2018

Reference:

https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/


**Environment:**

- Python 3.6
- Keras-gpu 2.2.0
- matplotlib 2.2.2
- pillow 5.1.0

# Build a base model

In [1]:
import keras
from keras.applications import VGG16
from keras import models
from keras import layers
from keras.layers import *
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image   # for load_image
import tensorflow as tf

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

Using TensorFlow backend.


# Freeze layers from pretrained model

In [2]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000002A48C2DAB00> False
<keras.layers.convolutional.Conv2D object at 0x000002A4934FD748> False
<keras.layers.convolutional.Conv2D object at 0x000002A49351CE10> False
<keras.layers.pooling.MaxPooling2D object at 0x000002A49351CB00> False
<keras.layers.convolutional.Conv2D object at 0x000002A4934FD9E8> False
<keras.layers.convolutional.Conv2D object at 0x000002A494503F28> False
<keras.layers.pooling.MaxPooling2D object at 0x000002A49451EB70> False
<keras.layers.convolutional.Conv2D object at 0x000002A494534978> False
<keras.layers.convolutional.Conv2D object at 0x000002A49455B630> False
<keras.layers.convolutional.Conv2D object at 0x000002A49454CC50> False
<keras.layers.pooling.MaxPooling2D object at 0x000002A494581A90> False
<keras.layers.convolutional.Conv2D object at 0x000002A494591A58> False
<keras.layers.convolutional.Conv2D object at 0x000002A4945BE828> False
<keras.layers.convolutional.Conv2D object at 0x000002A4945AAF28> False
<ker

# Create a new model

In [3]:
# from keras import models
# from keras import layers
# from keras import optimizers
 
targetClassNumber = 11   
    
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(targetClassNumber, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 11)                11275     
Total params: 40,417,099
Trainable params: 32,781,835
Non-trainable params: 7,635,264
_________________________________________________________________


# Setup the data generators

In [4]:
train_dir = './train'
validation_dir = './validation'
image_size = 224
 
# nTrain = 600
# nVal = 150

# from keras.preprocessing.image import ImageDataGenerator
# import numpy as np
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 20
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 33 images belonging to 11 classes.
Found 22 images belonging to 11 classes.


# Save image labels

### as a pickle file to look up during prediction step

In [5]:
import pickle

# Label dictionary of train images
class_dictionary = train_generator.class_indices
print('Train Label Dictionary : \n',class_dictionary)

with open('labels.pickle', 'wb') as f:
    pickle.dump(class_dictionary, f)

Train Label Dictionary : 
 {'Benjamin': 0, 'Damarcus': 1, 'Frank': 2, 'Lokesh': 3, 'Misael': 4, 'Sohail': 5, 'aaron': 6, 'albert': 7, 'brian': 8, 'mooyoung-lee': 9, 'vivek': 10}


# Create a TensorBoard Logger

In [6]:
# logger = keras.callbacks.TensorBoard(
#     log_dir='logs',
#     histogram_freq=5,
#     write_graph=True
# )

# Train the model


In [7]:
# # Compile the model
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.RMSprop(lr=1e-4),
#               metrics=['acc'])
# # Train the model
# history = model.fit_generator(
#       train_generator,
#       steps_per_epoch=train_generator.samples/train_generator.batch_size ,
#       epochs=24,
#       validation_data=validation_generator,
#       validation_steps=validation_generator.samples/validation_generator.batch_size,
#       verbose=1)
 
# Save the model
# model.save('faceID_VGG16.h5')

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=24,
      verbose=1)

Epoch 1/24
2/1 [====================================] - 7s 4s/step - loss: 5.2684 - acc: 0.0535
Epoch 2/24
2/1 [====================================] - 1s 496ms/step - loss: 2.7792 - acc: 0.1250
Epoch 3/24
2/1 [====================================] - 1s 628ms/step - loss: 2.4184 - acc: 0.2589
Epoch 4/24
2/1 [====================================] - 1s 572ms/step - loss: 2.1956 - acc: 0.3124
Epoch 5/24
2/1 [====================================] - 1s 496ms/step - loss: 2.0947 - acc: 0.1965
Epoch 6/24
2/1 [====================================] - 1s 498ms/step - loss: 1.6163 - acc: 0.4196
Epoch 7/24
2/1 [====================================] - 1s 598ms/step - loss: 1.5104 - acc: 0.5268
Epoch 8/24
2/1 [====================================] - 1s 589ms/step - loss: 1.1207 - acc: 0.5357
Epoch 9/24
2/1 [====================================] - 1s 555ms/step - loss: 1.4552 - acc: 0.4821
Epoch 10/24
2/1 [====================================] - 1s 522ms/step - loss: 0.8491 - acc: 0.8392
Epoch 11/24


# Google Compatible Model

In [9]:
model_builder = tf.saved_model.builder.SavedModelBuilder("faceID_VGG16_gMLapi")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

# save structure and weight
model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'faceID_VGG16_gMLapi\\saved_model.pb'


b'faceID_VGG16_gMLapi\\saved_model.pb'